## Overpass API 

works with https://github.com/mocnik-science/osm-python-tools/ library

categories are presented for example here https://github.com/GIScience/openpoiservice/blob/master/categories_docker.yml

In [1]:
import pandas as pd
import time
from OSMPythonTools.overpass import overpassQueryBuilder
from OSMPythonTools.overpass import Overpass
overpass = Overpass(waitBetweenQueries = 50)

In [3]:
custom_bbox = [48.1, 16.3, 48.3, 16.5]

In [4]:
def poi_request(item_type, search_query):
    # query built
    query = overpassQueryBuilder(bbox=custom_bbox, elementType='node', 
                                     selector='"{}"="{}"'.format(str(item_type), str(search_query)), out='body')
    result = overpass.query(query, timeout=100000)

    # result in json
    result = result.toJSON()['elements']
    
    # separate tags
    for row in result:
        row.update(row['tags'])

    df = pd.DataFrame(result)
    return df

In [2]:
# lists of items to request

tourism = ['hotel', 'motel']
amenity = ['library', 'museum', 'bank', 'hospital', 'cafe', 'fast_food', 'pub', 'restaurant']
shop = ['shoes', 'alcohol', 'bakery', 'cheese', 'tobacco']

In [8]:
# request for items in item category
for search_query in amenity:
    df = poi_request('amenity', search_query)
    time.sleep(100)

    df.to_csv("./output/{}.csv".format(str(search_query)))

[overpass] downloading data: [timeout:100000][out:json];(node["amenity"="library"](48.1,16.3,48.3,16.5);); out body;
[overpass] downloading data: [timeout:100000][out:json];(node["amenity"="museum"](48.1,16.3,48.3,16.5);); out body;
[overpass] downloading data: [timeout:100000][out:json];(node["amenity"="bank"](48.1,16.3,48.3,16.5);); out body;
[overpass] downloading data: [timeout:100000][out:json];(node["amenity"="hospital"](48.1,16.3,48.3,16.5);); out body;
[overpass] downloading data: [timeout:100000][out:json];(node["amenity"="cafe"](48.1,16.3,48.3,16.5);); out body;
[overpass] downloading data: [timeout:100000][out:json];(node["amenity"="fast_food"](48.1,16.3,48.3,16.5);); out body;
[overpass] downloading data: [timeout:100000][out:json];(node["amenity"="pub"](48.1,16.3,48.3,16.5);); out body;
[overpass] downloading data: [timeout:100000][out:json];(node["amenity"="restaurant"](48.1,16.3,48.3,16.5);); out body;
